In [3]:
#Importing Libraries

#import Client ID and Client Secret ID from config.py
from config import cid, secret

#Importing dependencies (pandas, json, requests, numpy, matplotlib)
import pandas as pd
import json
import requests
import numpy as np
import matplotlib.pyplot as plt

#Spotipy APU for Spotify (to pull data)
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager
=
client_credentials_manager)

In [11]:
# Define lists to add to csv
artist_name = []
track_name = []
popularity = []
track_id = []
audio_features = []

# Loop through the 1000 most popular songs associated with the year 2023
for i in range(0,1000,50):
    track_results = sp.search(q='year:2023', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
        audio_features.append(sp.audio_features(t['id']))

In [15]:
# Make a dataframe with the track name, artist name, track id, and popularity of the track
track_df = pd.DataFrame({'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'popularity' : popularity})
print(track_df.shape)
track_df.head()

# Make a dataframe with the trackID and the audio features of each track
features_df = pd.DataFrame({'track_id': track_id, 'AudioFeatures' : audio_features})

# Save both the dataframes to csv files
track_df.to_csv('track_df.csv')
features_df.to_csv('features_df.csv')

(1000, 4)
